In [2]:
import xarray as xr

# Load the SINMOD NetCDF file
sinmod_file = '/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc'
ds = xr.open_dataset(sinmod_file)

# Print a summary of the dataset
print(ds)

<xarray.Dataset> Size: 157GB
Dimensions:          (time: 365, dummySampleDim: 6, zc: 50, xc: 950, yc: 555)
Coordinates:
    time             (time, dummySampleDim) int32 9kB ...
  * xc               (xc) float32 4kB 800.0 1.6e+03 ... 7.592e+05 7.6e+05
  * yc               (yc) float32 2kB 800.0 1.6e+03 ... 4.432e+05 4.44e+05
  * zc               (zc) float32 200B 3.0 6.0 10.0 ... 2.5e+03 2.75e+03 3e+03
Dimensions without coordinates: dummySampleDim
Data variables:
    grid_mapping     int32 4B ...
    LayerDepths      (zc) float32 200B ...
    depth            (yc, xc) float32 2MB ...
    DXxDYy           (yc, xc) float32 2MB ...
    u_velocity       (time, zc, yc, xc) float32 38GB ...
    v_velocity       (time, zc, yc, xc) float32 38GB ...
    elevation        (time, yc, xc) float32 770MB ...
    temperature      (time, zc, yc, xc) float32 38GB ...
    salinity         (time, zc, yc, xc) float32 38GB ...
    ice_thickness    (time, yc, xc) float32 770MB ...
    ice_compactness  (time

In [6]:
import rasterio

# Open a TIFF file to extract resolution and bounds

tif_file = 'raw_data/EMOD-tifs/C5_2022_mean_UTM_32_resam.tif'

with rasterio.open(tif_file) as tif:
    target_crs = tif.crs  # EPSG:32632
    target_bounds = tif.bounds
    target_transform = tif.transform
    target_resolution = tif.res
    print(f"Target CRS: {target_crs}, Bounds: {target_bounds}, Resolution: {target_resolution}")


Target CRS: EPSG:32632, Bounds: BoundingBox(left=109798.28607731192, bottom=6812340.081526826, right=830804.4917635358, top=7965849.508304196), Resolution: (111.06072176312752, 87.50640470166668)


In [9]:
from pyproj import CRS

def obtain_sinmod_crs(PhysStates_data):
    # Access grid mapping attributes from the dataset
    grid_mapping = PhysStates_data['grid_mapping']  # Replace 'grid_mapping' with the correct variable name if different
    grid_attrs = grid_mapping.attrs  # Get the attributes of the grid mapping

    # Print horizontal resolution if available
    horizontal_resolution = grid_attrs.get('horizontal_resolution', 'unknown')
    print(f"\nHorizontal resolution: {horizontal_resolution} meters")

    # Construct the CRS using the attributes
    crs_sinmod = CRS.from_proj4(
        f"+proj=stere "
        f"+lat_0={grid_attrs['latitude_of_projection_origin']} "
        f"+lat_ts={grid_attrs['standard_parallel']} "
        f"+lon_0={grid_attrs['straight_vertical_longitude_from_pole']} "
        f"+x_0={grid_attrs['false_easting']} "
        f"+y_0={grid_attrs['false_northing']} "
        f"+a={grid_attrs['semi_major_axis']} "
        f"+b={grid_attrs['semi_minor_axis']} "
        f"+units=m +no_defs"
    )

    print(f"\nDetermined SINMOD CRS: {crs_sinmod}")
    return crs_sinmod

midnor_crs = obtain_sinmod_crs(ds)

# Attach the CRS to the SINMOD dataset
ds = ds.rio.write_crs(midnor_crs)


Horizontal resolution: 800.0 meters

Determined SINMOD CRS: +proj=stere +lat_0=90.0 +lat_ts=60.0 +lon_0=58.0 +x_0=2544800.0 +y_0=1918800.0 +a=6370000.0 +b=6370000.0 +units=m +no_defs +type=crs


In [12]:
import rioxarray
from rasterio.enums import Resampling

# Reproject to match the TIFF CRS
ds_reprojected = ds.rio.reproject(
    dst_crs=target_crs.to_string(),
    resolution=target_resolution,  # Match TIFF resolution
    resampling=Resampling.bilinear  # Use 'nearest' for categorical variables
)

# Inspect reprojected data
print(ds_reprojected)

TooManyDimensions: Only 2D and 3D data arrays supported. Data variable: u_velocity

In [21]:
import rasterio

with rasterio.open('raw_data/EMOD-tifs/C5_slope.tif') as dataset:
    print(dataset.profile)
    data = dataset.read(1)

    dimensions = data.shape
    num_grids = data.size
    print(f"Dimensions: {dimensions}")
    print(f"Number of grids: {num_grids}")

slope = rasterio.open('raw_data/EMOD-tifs/C5_slope.tif')

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028230607370965e+38, 'width': 6491, 'height': 13181, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(80.0, 0.0, 210863.54288175795,
       0.0, -80.0, 7914483.248744318), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
Dimensions: (13181, 6491)
Number of grids: 85557871


In [49]:
from netCDF4 import Dataset

# Cut down each tif to the same grid boundaries as MidNor data
# 1. Get the grid boundaries of MidNor data
filename_PhysStates = '/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_jan_jun.nc'

midnor_PhysStates_data = Dataset(filename_PhysStates, 'r')

# # Looking at our list of available variables
# variables_list = list(midnor_PhysStates_data.variables.keys())
# for variable in variables_list:
#     units = midnor_PhysStates_data.variables[variable].units if 'units' in midnor_PhysStates_data.variables[variable].ncattrs() else 'No units'
#     print(f"{variable:<20} {units}")

# Get the corner points from gridLats and gridLons
gridLats = midnor_PhysStates_data.variables['gridLats'][:]
gridLons = midnor_PhysStates_data.variables['gridLons'][:]

corner_points = {
    'bottom_left': (gridLats[0, 0], gridLons[0, 0]),
    'bottom_right': (gridLats[0, -1], gridLons[0, -1]),
    'top_left': (gridLats[-1, 0], gridLons[-1, 0]),
    'top_right': (gridLats[-1, -1], gridLons[-1, -1])
}

# print("Corner Points:")
# for corner, (lat, lon) in corner_points.items():
#     print(f"{corner}: Latitude = {lat}, Longitude = {lon}")

# Get the corner coordinates in native coordinates using xc and yc
xc = midnor_PhysStates_data.variables['xc'][:]
yc = midnor_PhysStates_data.variables['yc'][:]

native_corner_points = {
    'bottom_left': (xc[0], yc[0]),
    'bottom_right': (xc[0], yc[-1]),
    'top_left': (xc[-1], yc[0]),
    'top_right': (xc[-1], yc[-1])
}

print("\nNative Corner Points:\n")
for corner, (x, y) in native_corner_points.items():
    print(f"{corner}: x = {x}, y = {y}")


Native Corner Points:

bottom_left: x = 800.0, y = 800.0
bottom_right: x = 800.0, y = 444000.0
top_left: x = 760000.0, y = 800.0
top_right: x = 760000.0, y = 444000.0


In [54]:
from pyproj import CRS, Transformer
import numpy as np
from rasterio import warp

def obtain_sinmod_crs(PhysStates_data):
    # Print grid mapping to see details, particularly the horizontal resolution
    grid_mapping = PhysStates_data.variables['grid_mapping']
    print(grid_mapping)

    horizontal_resolution = grid_mapping.getncattr('horizontal_resolution')
    print(f"\nHorizontal resolution: {horizontal_resolution} meters")

    crs_sinmod = CRS.from_proj4(f"+proj=stere "
                                f"+lat_0={grid_mapping.latitude_of_projection_origin} "
                                f"+lat_ts={grid_mapping.standard_parallel} "
                                f"+lon_0={grid_mapping.straight_vertical_longitude_from_pole} "
                                f"+x_0={grid_mapping.false_easting} "
                                f"+y_0={grid_mapping.false_northing} "
                                f"+a={grid_mapping.semi_major_axis} "
                                f"+b={grid_mapping.semi_minor_axis} "
                                f"+units=m +no_defs")
    
    print(f"\n Determined SINMOD CRS: {crs_sinmod}")
    return crs_sinmod

def transform_coral_coordinates(coral_data, crs_sinmod):
    # Define the Coordinate Reference Systems (CRS) for the coral data
    crs_wgs84 = CRS.from_epsg(4326)  # WGS84 (lat/lon coordinates)

    # Create a transformer to transform from wgs84 (geographic) to SINMOD
    transformer_wgs84_to_sinmod = Transformer.from_crs(crs_wgs84, crs_sinmod, always_xy=True)

    # Transform coral coordinates from WGS84 to SINMOD
    coral_x, coral_y = transformer_wgs84_to_sinmod.transform(coral_data['lengdegrad'], coral_data['breddegrad'])

    # Save coral_x and coral_y as new columns called x and y
    coral_data.loc[:, 'x'] = coral_x
    coral_data.loc[:, 'y'] = coral_y

    return coral_data

def transform_tif_coordinates(tif_data, crs_sinmod, transform):
    # Define the Coordinate Reference Systems (CRS) for the tif data
    crs_tif = CRS.from_epsg(32632)  # UTM Zone 32N

    # Create a transformer to transform from UTM Zone 32N to SINMOD
    transformer_tif_to_sinmod = Transformer.from_crs(crs_tif, crs_sinmod, always_xy=True)

    # Get the coordinates of the tif data
    rows, cols = tif_data.shape
    x_coords, y_coords = rasterio.transform.xy(transform, np.arange(rows), np.arange(cols), offset='center')

    # Transform tif coordinates from UTM Zone 32N to SINMOD
    tif_x, tif_y = transformer_tif_to_sinmod.transform(x_coords, y_coords)

    return tif_x, tif_y

def transform_raster_to_crs(input_tif, output_tif, target_crs):
    """
    Transforms a GeoTIFF file's coordinates to a specified CRS.
    
    Parameters:
        input_tif (str): Path to the input GeoTIFF file.
        output_tif (str): Path to save the transformed GeoTIFF file.
        target_crs (str): Target CRS in WKT or EPSG format (e.g., 'EPSG:3413').
    """
    with rasterio.open(input_tif) as src:
        # Define the transformer
        transformer = Transformer.from_crs(src.crs, target_crs, always_xy=True)

        # Transform the bounding box coordinates
        bounds = src.bounds
        corners = [
            (bounds.left, bounds.bottom),  # Bottom-left
            (bounds.right, bounds.top)    # Top-right
        ]
        transformed_corners = [transformer.transform(x, y) for x, y in corners]
        print(f"Transformed corners: {transformed_corners}")

        # Get the new bounds
        new_left, new_bottom = transformed_corners[0]
        new_right, new_top = transformed_corners[1]

        # Update the transform and metadata for the new CRS
        new_transform, new_width, new_height = rasterio.warp.calculate_default_transform(
            src.crs, target_crs, src.width, src.height, left=new_left, bottom=new_bottom, right=new_right, top=new_top
        )

        # Update metadata
        new_meta = src.meta.copy()
        new_meta.update({
            "crs": target_crs,
            "transform": new_transform,
            "width": new_width,
            "height": new_height
        })

        # Write the transformed raster to the output file
        with rasterio.open(output_tif, "w", **new_meta) as dst:
            rasterio.warp.reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=new_transform,
                dst_crs=target_crs,
                resampling=rasterio.warp.Resampling.nearest
            )


In [ ]:
midnor_crs = obtain_sinmod_crs(midnor_PhysStates_data)

transform_raster_to_crs('raw_data/EMOD-tifs/C5_slope.tif', 'processed_data/tifs/C5_slope_transformed.tif', midnor_crs)

<class 'netCDF4._netCDF4.Variable'>
int32 grid_mapping()
    grid_mapping_name: polar_stereographic
    straight_vertical_longitude_from_pole: 58.0
    horizontal_resolution: 800.0
    latitude_of_projection_origin: 90.0
    longitude_of_projection_origin: 58.0
    standard_parallel: 60.0
    origoRef: [0. 0.]
    semi_minor_axis: 6370000.0
    semi_major_axis: 6370000.0
    false_easting: 2544800.0
    false_northing: 1918800.0
    scale_factor_at_projection_origin: 1.0
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of -2147483647 used

Horizontal resolution: 800.0 meters

 Determined SINMOD CRS: +proj=stere +lat_0=90.0 +lat_ts=60.0 +lon_0=58.0 +x_0=2544800.0 +y_0=1918800.0 +a=6370000.0 +b=6370000.0 +units=m +no_defs +type=crs
Transformed corners: [(111322.95730004925, 181891.28435119358), (1214916.3274095315, 471715.03834121255)]


In [57]:
slope_transformed = rasterio.open('raw_data/EMOD-tifs/C5_slope_transformed.tif')

slope_transformed.width

12341